In [1]:
#### GOOGLE DRIVE SPECIFIC ##########################
# Make sure that you have GPU selected in the Runtime
# If you do, will print Found GPU at: /device:GPU:0
# Else go to Runtime -> Change Runtime Type

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
# CODE SNIPPET TO ACCESS THE FILES IN GOOGLE DRIVE (GO TO BROWSER AND VERIFY)
# THEN YOU CAN ACCESS THE FILES ON LEFT SIDEBAR (copy path)
# (https://colab.research.google.com/drive/1srw_HFWQ2SMgmWIawucXfusGzrj1_U0q#scrollTo=H4SJ-tGNkOeY)

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
# drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
# !ls "/content/drive/My Drive"


Mounted at /content/drive


In [0]:
# googlepath exists for googledrive finding the files. 
googlepath = "drive/My Drive/SeniorDesign19"

# folder that holds the real data 
foldernamepath = googlepath + "/Data/32BY32"

testdatapath = googlepath + "/Data/32BY32TestSet"

# saved model path
modelpath = googlepath + "/TrafficSignGAN/cnnModel/"

# path to generated SINGLE images from GAN
generatedpath = googlepath + "/TrafficSignGAN/trainedModelsandOutputs/"

In [0]:
# Control whether to include generated images into CNN training
includeGenerated = True

# Which model to use. 
# Model 1 is from Keras CIFAR-10 CNN 
# Model 2 is Basic Lecun Network
# Model 3 is Keras ConvNet
modelNumber = 1

In [5]:

import keras

from keras import optimizers, regularizers
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout, Activation
from keras.callbacks import LearningRateScheduler, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping

from keras.layers import Input, Reshape
from keras.layers import BatchNormalization, Conv2DTranspose
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model, load_model
from keras.optimizers import Adam


import sys
import numpy as np
import cv2
import os
import re

import numpy as np

import random


Using TensorFlow backend.


In [0]:
# Testing on Different Models
def build_model():
  ######################
    
    if modelNumber == 1:       
        model = Sequential()
        model.add(Conv2D(32, (3, 3), padding='same',
                          input_shape=(32,32,3)))
        model.add(Activation('relu'))
        model.add(Conv2D(32, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Conv2D(64, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(Conv2D(64, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(512))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))
        model.add(Dense(10))
        model.add(Activation('softmax'))
    
        # initiate RMSprop optimizer
        opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

        # Let's train the model using RMSprop
        model.compile(loss='categorical_crossentropy',
                      optimizer=opt,
                      metrics=['accuracy'])
        
    elif modelNumber == 2:
      
        model = Sequential()
        model.add(Conv2D(6, (5, 5), padding='valid', activation = 'relu', kernel_initializer='he_normal', input_shape=(32,32,3)))
        model.add(MaxPooling2D((2, 2), strides=(2, 2)))
        model.add(Conv2D(16, (5, 5), padding='valid', activation = 'relu', kernel_initializer='he_normal'))
        model.add(MaxPooling2D((2, 2), strides=(2, 2)))
        model.add(Flatten())
        model.add(Dense(120, activation = 'relu', kernel_initializer='he_normal'))
        model.add(Dense(84, activation = 'relu', kernel_initializer='he_normal'))
        model.add(Dense(10, activation = 'softmax', kernel_initializer='he_normal'))
        sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    
    elif modelNumber == 3:
        model = Sequential()

        model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)))
        model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(1024, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(10, activation='softmax'))

        # Compile the model
        model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(lr=0.0001, decay=1e-6),
                      metrics=['accuracy'])
        
    else:
        raise Exception("Model Number is not valid!")
        
    return model

In [0]:
# lower learning rate as we increase training epochs
def scheduler(epoch):
    if epoch < 100:
        return 0.01
    if epoch < 150:
        return 0.005
    return 0.001
  
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003        
    return lrate

In [8]:
### Getting the Data and Preprocessing

## MODIFIDED TO TAKE IMAGES FROM FOLDER########
img_names = os.listdir(foldernamepath)

## Get the real images of the 10 classes we are using
# Could preprocess once and just remove the other images as well...
# But let's keep those images there, for now.

realClassesSet = set([28, 54, 3, 5, 55, 35, 7, 30, 16, 11])

# Need to map classesSet from 0-9 
classMap = {}
for index, i in enumerate(realClassesSet):
  classMap[i] = index
  if index == 10:
    print("failed")


selectedImages = []

for img in img_names:
  m = img.split('_')[0]
#   m = re.findall(r'[0-9]+', img)
  classNum = int(m)
  if classNum in realClassesSet:
    selectedImages.append(img)

img_names = selectedImages

# shuffle these names
random.shuffle(img_names)


# Need to also create the y_labels for each image
y_train = []
x = []
checkUp = 0
realLength = len(img_names)

for img_name in img_names:
  checkUp += 1
  if checkUp % 500 == 0:
    print("At Real Image: ", checkUp, " of total images: ", realLength)
  if img_name is not None:
    im = np.asarray(cv2.imread(foldernamepath + "/" + img_name, cv2.IMREAD_COLOR))
    if im.shape[0] == 32 and im.shape[1] == 32:
      x.append(im)
      classLabel = int(img_name.split('_')[0])
      y_train.append(classMap[classLabel])
      
currLength = len(x)
print("# of Real Images: ", currLength)


if includeGenerated:
    classDir = []
    # also the generated images for each class          
    for classNum in realClassesSet:
      classDir.extend(os.listdir(generatedpath + "class_{0}/class_{0}_singleImages".format(classNum)))

    random.shuffle(classDir)

    generatedLength = len(classDir)

    checkUp = 0
    for img_name in classDir:
      checkUp += 1
      if checkUp % 500 == 0:
        print("At Generated Image: ", checkUp, " of total images: ", generatedLength)
      if img_name is not None:
        classLabel = int(img_name.split('_')[1])
        im = np.asarray(cv2.imread(generatedpath + "class_{0}/class_{0}_singleImages".format(classLabel) 
                                   + "/" + img_name, cv2.IMREAD_COLOR))
        if im.shape[0] == 32 and im.shape[1] == 32:
          x.append(im)
          y_train.append(classMap[classLabel])

    print("# of Generated Images: ", len(x) - currLength)
    print("Training on # of Real + Generated Images: ", len(x))


# prepare the test set
checkUp = 0
test_names = os.listdir(testdatapath)
testLength = len(test_names)
X_test_original = []
y_test_original = []

for img_name in test_names:
  checkUp += 1
  if checkUp % 500 == 0:
    print("At Test Image: ", checkUp, " of total images: ", testLength)
  if img_name is not None:
    im = np.asarray(cv2.imread(testdatapath + "/" + img_name, cv2.IMREAD_COLOR))
    if im.shape[0] == 32 and im.shape[1] == 32:
      X_test_original.append(im)
      classLabel = int(img_name.split('_')[0])
      y_test_original.append(classMap[classLabel])

    
y_train_original = y_train
X_train_original = np.asarray(x)  
print ("Training shape: {}".format(X_train_original.shape))

X_test_original = np.asarray(X_test_original)   
print ("Testing shape: {}".format(X_test_original.shape))


At Real Image:  500  of total images:  2124
At Real Image:  1000  of total images:  2124
At Real Image:  1500  of total images:  2124
At Real Image:  2000  of total images:  2124
# of Real Images:  2124
At Generated Image:  500  of total images:  4500
At Generated Image:  1000  of total images:  4500
At Generated Image:  1500  of total images:  4500
At Generated Image:  2000  of total images:  4500
At Generated Image:  2500  of total images:  4500
At Generated Image:  3000  of total images:  4500
At Generated Image:  3500  of total images:  4500
At Generated Image:  4000  of total images:  4500
At Generated Image:  4500  of total images:  4500
# of Generated Images:  4500
Training on # of Real + Generated Images:  6624
At Test Image:  500  of total images:  772
Training shape: (6624, 32, 32, 3)
Testing shape: (772, 32, 32, 3)


In [0]:
# Training the CNN Model on the Images

# Make first 500 of REAL and SHUFFLED images my 
# validation data

threshold = 500

# Doing this so I can rerun this code on the fly and not have it mess up
y_val = y_train_original[:threshold]
y_train = y_train_original[threshold:]
y_test = y_test_original[0:]

X_val = X_train_original[:threshold]
X_train = X_train_original[threshold:]
X_test = X_test_original[0:]

y_train = keras.utils.to_categorical(y_train, 10)
y_val = keras.utils.to_categorical(y_val, 10)
y_test = keras.utils.to_categorical(y_test, 10)

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')

# Not sure this is best way to normalize.
X_train /= 255.0
X_val /= 255.0
X_test /= 255.0

In [10]:
epochs = 100
accuracies = []

# Get the average accuracy over 10 runs
for i in range(10):
# build network
    model = build_model()
#     print(model.summary())
    
    if modelNumber == 2:
        # set callback
        tb_cb = TensorBoard(log_dir='./lenet', histogram_freq=0)
        change_lr = LearningRateScheduler(scheduler)
        cbks = [change_lr,tb_cb]

        # start train
        model.fit(X_train, y_train,
                  batch_size=128,
                  epochs=epochs,
                  callbacks=cbks,
                  validation_data=(X_val, y_val),
                  shuffle=False,
                  verbose=0)
    elif modelNumber == 3:
      
        model.fit(X_train, y_train,
          batch_size=128,
          shuffle=False,
          epochs=epochs,
          validation_data=(X_val, y_val),
          callbacks=[EarlyStopping(min_delta=0.001, patience=3)],
          verbose=0)
        
    else:
        model.fit(X_train, y_train,
                  batch_size=128,
                  epochs=epochs,
                  validation_data=(X_val, y_val),
                  shuffle=False,
                  verbose=0)


    # Score trained model.
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Run {}".format(i+1))
    print("--------------------------------------------")
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])

    print("Threshold: ", threshold)
    print("Epochs: ", epochs)
    print("Include Generated?: ", includeGenerated)
    print("Model Number: ", modelNumber)
    print("--------------------------------------------")
    print()
    
    accuracies.append(scores[1])
    
print("Model {}; Include Generated? {};\navg acc of 10 runs: {};\nstd dev: {}".format(
    modelNumber, includeGenerated, np.mean(accuracies), np.std(accuracies)))
print()
    
# save model
# model.save(modelpath + 'lenet.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Run 1
--------------------------------------------
Test loss: 1.353193498612261
Test accuracy: 0.844559585492228
Threshold:  500
Epochs:  100
Include Generated?:  True
Model Number:  1
--------------------------------------------

Run 2
--------------------------------------------
Test loss: 1.3613188819992332
Test accuracy: 0.8367875647668394
Threshold:  500
Epochs:  100
Include Generated?:  True
Model Number:  1
--------------------------------------------

Run 3
--------------------------------------------
Test loss: 1.5376259586178993
Test accuracy: 0.8264248704663213
Threshold:  500
Epochs:  100
Include Generated?:  True
Model Number:  1
--------------------------------------------

Run 4
--------------------------------------------
Test loss: 1.4141